# Tool Call LLMs
Some of the more exciting use cases for RC is when you want to cede control of the decision making process to the LLM. RC has a suite of tools that you can use to make this process much simpler. Check out an example below. 

In [6]:
from typing import Dict, Any, Set, Type

from railtownai_rc.nodes import Node
from typing_extensions import Self

import railtownai_rc as rc
from railtownai_rc.llm import ModelBase, Tool, Parameter

In [7]:
class HarshCritic(rc.nodes.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a harsh critic of the world and you should analyze the given statement and provide a harsh critique of it."
    
    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Harsh Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> Tool:
        return Tool(
            name="Harsh_Critic",
            detail="A tool used to critique a statement harshly.",
            parameters={Parameter("analysis_detail", "string", "The thing you would like to analyze")}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> Self:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)
        
    

In [8]:
class PositiveCritic(rc.nodes.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a positive critic of the world and you should analyze the given statement and provide a positive critique of it."
    
    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Positive Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> Tool:
        return Tool(
            name="Positive_Critic",
            detail="A tool used to critique a statement positively.",
            parameters={Parameter("analysis_detail", "string", "The thing you would like to analyze")}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> Self:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)

In [9]:
class DeeperMeaningCritic(rc.nodes.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a critic of the world and you should analyze the given statement and provide a critique of it."
    
    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Deeper Meaning Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> Tool:
        return Tool(
            name="Deeper_Meaning_Critic",
            detail="A tool used to critique a statement.",
            parameters={Parameter("analysis_detail", "string", "The thing you would like to analyze",)}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> Self:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)
    
    

In [10]:
class Critic(rc.nodes.library.ToolCallLLM):
    @classmethod
    def system_message(cls) -> str:
        return "You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques before completing your answer."

    @classmethod
    def create_model(cls) -> ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")

    @classmethod
    def tool_details(cls) -> Set[Type[Node]]:
        return {HarshCritic, PositiveCritic, DeeperMeaningCritic}

    @classmethod
    def pretty_name(cls) -> str:
        return "Critic"

In [ ]:
template = ("I am writing a short story and I would like to analyze my introduction.\n"
            "\n"
            "Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding"
            " of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a"
            " magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better"
            " tomorrow.")

response = rc.run.run(Critic(
    message_history=rc.llm.MessageHistory([rc.llm.UserMessage(template)])),
    executor_config=rc.run.ExecutorConfig(timeout=50)
)

print(response.answer)

c:\Users\Logan\Documents\rc\.venv\lib\site-packages\railtownai_rc\nodes\nodes.py:109: UserWarning: You are using the default context. It will be empty.
  warnings.warn("You are using the default context. It will be empty.")
